In [ ]:
import datasets
import math
import numpy as np

import torch
import torch.nn as nn
from tqdm import tqdm

from copy import deepcopy
from transformer_lens import HookedTransformer
from transformer_lens import HookedTransformerConfig
from transformer_lens.utils import lm_cross_entropy_loss
from transformer_lens.utils import tokenize_and_concatenate

from transformer_lens import HookedTransformerConfig, HookedTransformer

DEVICE = 'cuda'

In [ ]:
from icl.language.model import get_model_cfg
from icl.language.utils import load_hf_checkpoint

model_cfgs = {}
model_cfgs[1] = get_model_cfg(num_layers=1)
model_cfgs[2] = get_model_cfg(num_layers=2)

model = HookedTransformer(model_cfgs[2])

In [ ]:
L2_models = {}

for step in tqdm(range(0, 50_001, 100)):
    L2_models[step] = load_hf_checkpoint(step, n_layers=2)

mean_k_score = L2_models[0].all_composition_scores('K')[0][:, 1].mean()

In [ ]:
from collections import defaultdict

def get_comp_scores(comp_type):
  scores = defaultdict(list)
  for step in range(0, 50_001, 100):
    all_layer_scores = L2_models[step].all_composition_scores(comp_type)[0].detach()
    for layer_0 in range(8):
      for layer_1 in range(8):
        score = all_layer_scores[layer_0][1][layer_1].item()
        scores[(layer_0, layer_1)].append(score)
  return scores

k_scores = get_comp_scores('K')